# OpenAI's Whisper Speech Recognition Model

![alt text](whisper.jpg "Intro")


Whisper ist der Name eines automatischen Spracherkennungssystems (ASR) von OpenAI. ASR-Systeme wandeln gesprochene Sprache in geschriebenen Text um. Sie werden in einer Vielzahl von Anwendungen eingesetzt, darunter Sprachassistenten, Transkriptionsdienste und mehr.

Whisper wurde auf einer großen Menge multilingualer und multitaskaler Daten aus dem Web trainiert. Es wurde entwickelt, um in verschiedenen Anwendungen und Kontexten eine hohe Genauigkeit bei der Spracherkennung zu bieten. 


## Install Whisper

In [2]:
! pip install git+https://github.com/openai/whisper.git -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.5.1 requires numpy<1.24,>=1.16.0, but you have numpy 1.25.2 which is incompatible.
pinecone-client 2.2.1 requires pyyaml>=5.4, but you have pyyaml 5.3.1 which is incompatible.
de-core-news-sm 3.1.0 requires spacy<3.2.0,>=3.1.0, but you have spacy 3.7.0 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


## Modell laden

- Das sollte ca. 140 MB runterladen.

In [1]:
import whisper
import numpy as np
model = whisper.load_model("base")


## Können wir die GPU nutzen?

Auf der GPU vs. CPU arbeiten. Im besten Fall steht dort device GPU, das hängt von eurem Laptop ab. 

In [3]:
model.device

## Define the Transcribe Function

Jetzt haben wir das Modell geladen und haben den Code. Dies ist die Funktion, die einen Audiodateipfad als Eingabe verwendet und den erkannten Text zurückgibt (und protokolliert, was seiner Meinung nach die Sprache ist).

In [27]:
CHUNK_LIM = 480000 # On my system....

def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audios = []

    if len(audio) <= CHUNK_LIM:
        audios.append(audio)
    else:
        for i in range(0, len(audio), CHUNK_LIM):
            chunk = audio[i:i + CHUNK_LIM]
            chunk_index = len(chunk)
            if chunk_index < CHUNK_LIM:
                padding = [0] * (CHUNK_LIM - chunk_index)
                array1 = np.array(chunk)
                array2 = np.array(padding)
                concat =  np.concatenate((array1, array2))
                chunk = concat.astype(np.float32)
            audios.append(chunk)

    results = ""

    for chunk in audios:

        # make log-Mel spectrogram and move to the same device as the model
        mel = whisper.log_mel_spectrogram(chunk).to(model.device)

        # decode the audio
        # bei M1 muss hier fp16 = False stehen, das müsst ihr nicht unbeding übernehmen
        options = whisper.DecodingOptions(fp16=False)
        result = whisper.decode(model, mel, options)
        results += result.text

    
    return results

## Test mit High Quality Audio

Ihr werdet feststellen, dass das zweite Transkript normalerweise nach 30 Sekunden abgeschnitten wird. Dies ist die Standardlänge. Diese kann erweitert werden. Siehe oben, mit chunking.

In [20]:
easy_text = transcribe("mary.mp3")
print(easy_text)

hard_text = transcribe("daisy_HAL_9000.mp3")
print(hard_text)

Detected language: en
Mary had a little lamb, its fleece was white as snow, and everywhere that Mary went, the lamb was sure to go.
Detected language: en
Tazy, Tazy, Tazy. Give me your answer to time after crazy all for the love of you. It won't be a stylish marriage


## Selbst ausprobieren mit dem Web UI Toolkit


In [21]:
! pip install gradio -q

In [16]:
import gradio as gr
import time

/Users/plotti/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Web Interface

After running this script, you should see two widgets below that you can use to record live audio and see the transcription, as described in the introduction.

In [17]:
gr.Interface(
    title = 'OpenAI Whisper ASR Gradio Web UI',
    fn=transcribe,
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath")
    ],
    outputs=[
        "textbox"
    ],
    live=True).launch()

/var/folders/cv/22t0yrj153n0r99j1psfrx6c0000gn/T/ipykernel_34797/1713949754.py:5: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Audio(source="microphone", type="filepath")
/var/folders/cv/22t0yrj153n0r99j1psfrx6c0000gn/T/ipykernel_34797/1713949754.py:5: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Audio(source="microphone", type="filepath")


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/plotti/miniconda3/lib/python3.9/site-packages/gradio/processing_utils.py:188: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Detected language: de


# Download a youtube video, transcode it.

In [30]:
#!yt-dlp "https://www.youtube.com/watch?v=ww0zvl18-Ys"

In [31]:
charlie = transcribe("Charlie Chaplins Rede an die Menschheit [ww0zvl18-Ys].webm")

In [32]:
charlie

'Es tut mir leid, aber ich möchte nun mal kein Herrscher der Welt sein, denn das liegt mir nicht. Ich möchte weder Herrchen noch irgendwie erobern, sondern jede Menschen helfen, wo immer ich kann, den Juden, den Heiden, den Fahrwigen, den Weißen. Jeder Mensch sollte dem anderen helfen, nur so verbessern wir die Welt. Wir sollten am Glück des anderen Teilhaben und nicht an der Verabschollen. Pass und Verachtung bringen uns niemals näher. Auf dieser Welt ist Platz genug für jeden und Mutter Erde ist reich genug, um jeden von uns Satz zu machen.Wir müssen es nur wieder zu leben lernen. Die Habki hat das Gute im Menschen verschützt. Und Missgunst hat die Seelen vergiftet und uns im Parade-Schritt zu verdärb und Blutschuld geführt. Wir haben die Geschwindigkeit entwickelt, aber innerlich sind wir stehen geblieben. Wir lassen Maschinen für uns arbeiten und sie denken auch für uns. Die Glugheit hat uns hochmütig werden lassen und unser Wissen kalt und hart. Wir sprechen zu viel und fühlen zu 

In [1]:
charlie = transcribe("tarczynski.webm")